# Task 5 #
==========

Итак... давай, наконец, выясним, повлияло ли посещение ленты новостей на поведение учащихся. Приступили ли они в итоге к работе над лабораторным заданием раньше? Помни, что у нас есть две подготовленные таблицы в базе данных: test и control. Мы выполним нечто, похожее на A/B-тестирование. Чтобы обнаружить эффект, нам нужно вычислить значение дельты (период времени между датой первого коммита и сроком сдачи лабораторного задания) до того момента, когда учащиеся впервые посетили страницу с лентой новостей, и после этого. Мы должны сделать то же самое и для контрольной группы.

Другими словами, каждый пользователь из тестовой таблицы имеет свою собственную временную метку для первого посещения новостной ленты. Мы хотим вычислить среднее значение дельты (разницу между датой первого коммита и сроком сдачи) до этой временной метки и после нее. Мы сделаем то же самое для пользователей в контрольной группе. Ты можешь сказать: «Но они вообще не посещали ленту новостей». Это так, и ранее мы решили использовать среднюю временную метку первого просмотра пользователями тестовой группы для пользователей контрольной группы.

Если значение дельты перед первым посещением ленты новостей значительно отличается от этого показателя после первого посещения в тестовой группе, и мы не видим аналогичного эффекта в контрольной группе, значит, создание страницы с новостной лентой было отличной идеей. Мы можем распространить эту практику на всю группу.

Подробное описание:

In [1]:
import sqlite3
import pandas as pd

## Создай соединение с базой данных с помощью библиотеки sqlite3. ##
----------

In [2]:
conn = sqlite3.connect("../data/checking-logs.sqlite")
cursor = conn.cursor()

In [3]:
# посмотреть на таблицы в базе данных
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('pageviews',), ('checker',), ('deadlines',), ('datamart',), ('control',), ('test',)]


In [4]:
cursor.execute("PRAGMA table_info(test);")
print(cursor.fetchall())

[(0, 'uid', 'TEXT', 0, None, 0), (1, 'labname', 'TEXT', 0, None, 0), (2, 'first_commit_ts', 'TIMESTAMP', 0, None, 0), (3, 'first_view_ts', 'TIMESTAMP', 0, None, 0)]


In [5]:
cursor.execute("PRAGMA table_info(control);")
print(cursor.fetchall())

[(0, 'uid', 'TEXT', 0, None, 0), (1, 'labname', 'TEXT', 0, None, 0), (2, 'first_commit_ts', 'TIMESTAMP', 0, None, 0), (3, 'first_view_ts', 'TIMESTAMP', 0, None, 0)]


## Используя только один запрос для каждой из групп, создай два датафрейма: test_results и control_results со столбцами time и avg_diff и только двумя строками. ##
-------

* times должно иметь значения before и after.
* avg_diff содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
* Учитываются только те пользователи, для которых имеются наблюдения и before, и after.
* Мы по-прежнему не используем лабораторное задание 'project1'.

In [6]:
# Создаем запрос для test
test_query = f""" SELECT time, AVG( avg_diff)
            FROM (
                    SELECT uid, labname,
                            AVG((strftime('%s', first_commit_ts) - deadlines)/60/60) AS  avg_diff,
                            CASE
                                WHEN ((strftime('%s', first_commit_ts)) - strftime('%s', first_view_ts)) < 0
                                THEN "before" ELSE "after"
                            END AS time
                    FROM test
                    LEFT JOIN deadlines 
                    ON test.labname = deadlines.labs
                    WHERE labname != "project1"
                    GROUP BY uid, time
                )
            WHERE uid IN (
                            SELECT uid
                            FROM (
                                    SELECT uid,
                                            CASE
                                                WHEN ((strftime('%s', first_commit_ts)) - strftime('%s', first_view_ts)) < 0
                                                THEN "before" ELSE "after"
                                            END AS time
                                    FROM test
                            LEFT JOIN deadlines 
                            ON test.labname = deadlines.labs
                            WHERE labname !="project1"
                            GROUP BY uid, time
                                    )
            GROUP BY uid
            HAVING COUNT(uid) == 2
                            )
GROUP BY time"""

In [7]:
# Создаем запрос для control
control_query = f""" SELECT time, AVG( avg_diff)
            FROM (
                    SELECT uid, labname,
                            AVG((strftime('%s', first_commit_ts) - deadlines)/60/60) AS  avg_diff,
                            CASE
                                WHEN ((strftime('%s', first_commit_ts)) - strftime('%s', first_view_ts)) < 0
                                THEN "before" ELSE "after"
                            END AS time
                    FROM control
                    LEFT JOIN deadlines 
                    ON control.labname = deadlines.labs
                    WHERE labname != "project1"
                    GROUP BY uid, time
                )
            WHERE uid IN (
                            SELECT uid
                            FROM (
                                    SELECT uid,
                                            CASE
                                                WHEN ((strftime('%s', first_commit_ts)) - strftime('%s', first_view_ts)) < 0
                                                THEN "before" ELSE "after"
                                            END AS time
                                    FROM control
                            LEFT JOIN deadlines 
                            ON control.labname = deadlines.labs
                            WHERE labname !="project1"
                            GROUP BY uid, time
                                    )
            GROUP BY uid
            HAVING COUNT(uid) == 2
                            )
GROUP BY time"""

In [8]:
# Создаем датафреймы
test_results = pd.read_sql_query(test_query, conn)
control_results = pd.read_sql_query(control_query, conn)
test_results
control_results

,time,AVG( avg_diff)
0,after,-99.322222
1,before,-98.033333


## Закрой соединение с базой данных. ##
------

In [9]:
# Закрываем соединение
conn.close()

In [10]:
print("Test results:")
print(test_results)
print("Control results:")
print(control_results)

Test results:
     time  AVG( avg_diff)
0   after      -99.523810
1  before      -66.047619
Control results:
     time  AVG( avg_diff)
0   after      -99.322222
1  before      -98.033333
